In [11]:
#imports 

import numpy as np
import pandas as pd 
import seaborn as sns 
import openpyxl
import pickle 

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb


In [2]:
df = pd.read_excel('C:\\Users\\Malavi\\Desktop\\RFISUBCODDES Predictor\\Ancillary Rev - 2015 - 2023.xlsx', header=2)
df

,Flight Date,Point of Sale,Channel,RFISUBCOD,RFISUBCODDES,RFICOD,REMARKS,Count,Sum of Revenue USD
0,2015-01-01,Sri Lanka,ATO/CTO/GSA/PSA,0BJ,UPGRADE,Air transportation,UPGRADE,2,1223.78
1,2015-01-01,Sri Lanka,Call centre,0BJ,UPGRADE,Air transportation,UPGRADE,61,13516.32
2,2015-01-01,Hong Kong,ATO/CTO/GSA/PSA,0IK,EXCESS WEIGHT,Baggage,EXCESS WEIGHT,4,384.23
3,2015-01-01,India Tamilnadu - Chennai,ATO/CTO/GSA/PSA,0IK,EXCESS WEIGHT,Baggage,EXCESS WEIGHT,26,1658.70
4,2015-01-01,India Trivandrum,ATO/CTO/GSA/PSA,0IK,EXCESS WEIGHT,Baggage,EXCESS WEIGHT,6,272.64
...,...,...,...,...,...,...,...,...,...
149997,2020-01-07,India Northern,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,2,4.06
149998,2020-01-07,Singapore,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,5,12.60
149999,2020-01-07,Sri Lanka,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,1,3.04
150000,2020-01-07,United Kingdom,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,5,25.08


In [3]:
# Convert Flight Date to datetime and extract Year & Month


df['Flight Date'] = pd.to_datetime(df['Flight Date'])
df['Year'] = df['Flight Date'].dt.year
df['Month'] = df['Flight Date'].dt.month

# Aggregate revenue by Year, Month, Point of Sale, RFISUBCODDES, and Count

df_main = df.groupby(['Year', 'Month', 'Point of Sale', 'RFISUBCODDES'])[['Sum of Revenue USD', 'Count']].sum().reset_index()

In [4]:
df_main

,Year,Month,Point of Sale,RFISUBCODDES,Sum of Revenue USD,Count
0,2015,1,Abu Dhabi & Al Ain,EXCESS WEIGHT,1952.19,46
1,2015,1,China South,EXCESS WEIGHT,6450.47,35
2,2015,1,China South,UPGRADE,201.07,1
3,2015,1,Dubai & Rest,EXCESS WEIGHT,53.12,1
4,2015,1,France,EXCESS WEIGHT,1401.73,18
...,...,...,...,...,...,...
11574,2020,1,United Kingdom,UPTO22LB/10KG - BAGGAGE,274.18,3
11575,2020,1,United States of America,EXCESS PIECE,832.96,13
11576,2020,1,United States of America,PRE PAID BAGGAGE,2942.79,36
11577,2020,1,United States of America,PRE-RESERVED SEAT ASSIGNMENT,3441.76,273


In [22]:
df_main.to_csv('output.csv', index=False)

In [5]:
df_main.isnull().sum()


Year                  0
Month                 0
Point of Sale         0
RFISUBCODDES          0
Sum of Revenue USD    0
Count                 0
dtype: int64

In [6]:
df_main.describe()

,Year,Month,Sum of Revenue USD,Count
count,11579.000000,11579.000000,11579.000000,11579.000000
mean,2017.633388,6.656102,4488.225894,50.664565
std,1.298135,3.533134,21668.340003,150.385518
min,2015.000000,1.000000,0.000000,1.000000
25%,2017.000000,4.000000,74.755000,2.000000
50%,2018.000000,7.000000,422.770000,7.000000
75%,2019.000000,10.000000,2522.040000,32.000000
max,2020.000000,12.000000,704615.740000,3589.000000


In [10]:
# Get the RFISUBCODDES that generates the highest revenue for each Month and Point of Sale
df_main= df_main.loc[df_main.groupby(['Month', 'Point of Sale'])['Sum of Revenue USD'].idxmax()]

In [12]:
# Encode categorical variables
le_pos = LabelEncoder()
le_rfisubcoddes = LabelEncoder()

df_main['Point of Sale'] = le_pos.fit_transform(df_main['Point of Sale'])
df_main['RFISUBCODDES'] = le_rfisubcoddes.fit_transform(df_main['RFISUBCODDES'])

In [14]:
# Define Features (X) and Target (y) 
X = df_main[['Year', 'Month', 'Point of Sale','Count']]
y = df_main['RFISUBCODDES']


In [15]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# Train XGBoost model
model = xgb.XGBClassifier(objective='multi:softmax', num_class=len(np.unique(y)), eval_metric='mlogloss', use_label_encoder=False)
model.fit(X_train, y_train)


C:\Users\Malavi\AppData\Roaming\Python\Python313\site-packages\xgboost\training.py:183: UserWarning: [11:51:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=14, ...)

In [17]:
# model accuracy
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")


Model Accuracy: 0.75


In [18]:
def predict_rfisubcoddes(year, month, point_of_sale, count=None):
    
    if count is None:
        count = df_main['Count'].mean()  
    
    
    pos_encoded = le_pos.transform([point_of_sale])[0]  
    
    
    prediction = model.predict([[year, month, pos_encoded, count]]) 
    
    
    return le_rfisubcoddes.inverse_transform(prediction)[0]


In [19]:
print(predict_rfisubcoddes( 2024,1, "Australia"))

EXCESS WEIGHT


In [ ]:
# Save the model
with open(r'models/oxgboost_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Save the label encoders
with open(r'models/label_encoder_pos.pkl', 'wb') as f:
    pickle.dump(le_pos, f)

with open(r'models/label_encoder_rfisubcoddes.pkl', 'wb') as f:
    pickle.dump(le_rfisubcoddes, f)